# Import Lib

In [1]:
import cv2
import os
import sys
sys.path.append("..")
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import json
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
# from tqdm import tqdm
from tqdm.notebook import tqdm
from scripts.function_test import resize_image
from scripts.augmentation_setup import custom_augment, complex_augmentation, test_augmentation, oversample_augmentation


# Get path

In [2]:
CSV_PATH = "D:\\Coding_practice\\_Data\\SIIM-ISIC_data\\train.csv"

data_frame = pd.read_csv(CSV_PATH)
data_frame

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


# Get all data from benign patients only

In [3]:
data_0 = data_frame.loc[np.where(data_frame.loc[:,'target']==0)]
data_0

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


# Get all data from malignant patients only

In [4]:
data_1 = data_frame.loc[np.where(data_frame.loc[:,'target']==1)]
data_1

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
91,ISIC_0149568,IP_0962375,female,55.0,upper extremity,melanoma,malignant,1
235,ISIC_0188432,IP_0135517,female,50.0,upper extremity,melanoma,malignant,1
314,ISIC_0207268,IP_7735373,male,55.0,torso,melanoma,malignant,1
399,ISIC_0232101,IP_8349964,male,65.0,torso,melanoma,malignant,1
459,ISIC_0247330,IP_3232631,female,65.0,lower extremity,melanoma,malignant,1
...,...,...,...,...,...,...,...,...
32969,ISIC_9955163,IP_7507212,male,55.0,upper extremity,melanoma,malignant,1
33000,ISIC_9963177,IP_1165806,male,70.0,torso,melanoma,malignant,1
33014,ISIC_9967383,IP_7887363,male,60.0,upper extremity,melanoma,malignant,1
33050,ISIC_9978107,IP_2860540,male,65.0,lower extremity,melanoma,malignant,1


# Get multiply constant

In [5]:
y = data_frame.loc[:,'target'].tolist()
weight = compute_class_weight('balanced', np.unique(y), y)
multiply_value = max(weight) / min(weight)
multiply_value

55.72260273972602

# Get image name from malignant patients 
### which is the class we need to over-sampling

In [6]:
IMG_PATH = "D:\\Coding_practice\\_Data\\SIIM-ISIC_data\\train"

imgs_list = []
for image in data_1.loc[:,'image_name']:
    imgs_list.append(image)

print(len(imgs_list))

584


In [8]:
round(multiply_value) * len(imgs_list)

32704

### Testing with getting image information

In [9]:
columns = 2
rows = 5
fig = plt.figure(figsize=(20,20))
image_choosen = os.path.join(IMG_PATH, random.choice(imgs_list) +".jpg")
img = cv2.imread(image_choosen)
img, _, _, _, _ = resize_image(img,min_dim=256, max_dim=512)
# fig.add_subplot(rows, columns, 1)
# plt.imshow(img)
# for i in range(2, columns * rows + 1):
    
#     # print(img.shape)
#     aug_img = oversample_augmentation(image=img)
#     fig.add_subplot(rows, columns, i)
#     plt.imshow(aug_img)
# plt.show()
over_sample = data_1.loc[np.where(data_frame.loc[:,'image_name']==random.choice(imgs_list))]
over_sample

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
26567,ISIC_8052769,IP_6630831,male,50.0,upper extremity,melanoma,malignant,1


<Figure size 1440x1440 with 0 Axes>

In [7]:
data_temp = data_frame
data_temp

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


In [8]:
EXPORT_PATH = "D:\\Coding_practice\\_Data\\SIIM-ISIC_data\\malignant_oversampling"
os.makedirs(EXPORT_PATH, exist_ok=True)
progress_bar = tqdm(imgs_list)
for iter ,image in enumerate(progress_bar):
    for i in range(round(multiply_value)-1):
        oversample_data = data_1.loc[np.where(data_frame.loc[:,'image_name']==image)]

        img_path =  os.path.join(IMG_PATH, image +".jpg")
        img = cv2.imread(img_path)
        img, _, _, _, _ = resize_image(img,min_dim=256, max_dim=512)
        oversample_img = oversample_augmentation(image=img)
        cv2.imwrite(os.path.join(EXPORT_PATH, image + f"_{i}.jpg"), oversample_img)
        
        oversample_data['image_name'] = oversample_data['image_name'].tolist()[0] + f"_{i}"
        data_temp = data_temp.append(oversample_data, ignore_index=True)
        
    progress_bar.set_description(f"Samples : {iter + 1}")
    progress_bar.update()
# over_sample

In [77]:
oversample_data = data_1.loc[np.where(data_frame.loc[:,'image_name']==image)]

oversample_data['image_name'].tolist()
oversample_data['image_name'] = oversample_data['image_name'].tolist()[0] +"_2"
oversample_data['image_name'].tolist()

['ISIC_0149568_2']

In [11]:
CSV_PATH_Oversampling = "D:\\Coding_practice\\_Data\\SIIM-ISIC_data\\train_oversampling.csv"
data_temp.to_csv(CSV_PATH_Oversampling, index=False)
